In [3]:
!pip install fasttext

In [3]:
import pandas as pd
import numpy as np
import os
import unicodedata
import torch
import json
import random
import fasttext
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score
import unicodedata

In [4]:
YelpZip_file_path = '/content/drive/MyDrive/CSE 573: Fake Reviews Project/YelpZip'

In [6]:
from google.colab import drive
drive.mount('/content/drive')
metadata = pd.read_csv(os.path.join(YelpZip_file_path, 'metadata'), sep= '\t',
                      names=['user_id', 'prod_id','rating','label', 'date'], header=None)

with open(os.path.join(YelpZip_file_path, 'reviewContent')) as r:
  content = r.readlines()

Mounted at /content/drive


In [7]:

number_of_samples = 30000

self_trained_model = fasttext.load_model('/content/drive/MyDrive/CSE 573: Fake Reviews Project/trained_YelpZip_model.bin')

def fasttext_emb(content, metadata,ft_model):
  dataset = []
  label = []
  for text, lab in zip(content[:number_of_samples], metadata[:number_of_samples].itertuples(index=True, name='Pandas')):
    sentence = text.strip('\n').split('\t')[-1]

    sentence = unicodedata.normalize("NFKD", sentence)
    
    sentence_vector = ft_model.get_sentence_vector(sentence)
    dataset.append(sentence_vector)

    if lab.label == -1:
      l = 0
    else: 
      l = 1
    label.append(l)
  
  return dataset, label

In [8]:
dataset,label = fasttext_emb(content, metadata,self_trained_model)
x_train, x_test, y_train, y_test = train_test_split(dataset, label, random_state=42,train_size=0.8) #train 80% test 20%


SVM


In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [2]:
svm = GridSearchCV(
    SVC(kernel="rbf"),
    param_grid = {'C': [1], 'gamma': np.logspace(5)},
    n_jobs= 12
)
svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)

NameError: ignored

In [ ]:
print("best param:", svm.best_estimator_)

best param: SVC(C=1, gamma=0.1)


In [ ]:
print("accuracy_score: ", accuracy_score(y_test,y_pred))
print("precision_score: ", precision_score(y_test,y_pred, pos_label=1))
print("recall_score: ", recall_score(y_test,y_pred, pos_label=1))
print("f1_score: ", f1_score(y_test,y_pred, pos_label=1))

accuracy_score:  0.8641666666666666
precision_score:  0.8641666666666666
recall_score:  1.0
f1_score:  0.9271345552078676


In [ ]:
print("accuracy_score: ", accuracy_score(y_test,y_pred))
print("precision_score: ", precision_score(y_test,y_pred, pos_label=0))
print("recall_score: ", recall_score(y_test,y_pred, pos_label=0))
print("f1_score: ", f1_score(y_test,y_pred, pos_label=0))

accuracy_score:  0.8641666666666666
precision_score:  0.0
recall_score:  0.0
f1_score:  0.0


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(all([i == 1 for i in y_pred]))

True
